In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/lib/kaggle/gcp.py
/kaggle/input/digit-recognizer/sample_submission.csv
/kaggle/input/digit-recognizer/train.csv
/kaggle/input/digit-recognizer/test.csv
/kaggle/working/__notebook_source__.ipynb


This code all my own, created from previous knowledge & school course work

In [38]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

In [57]:
train_data = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
X_train = train_data.drop("label",axis=1) / 255.0
y_train = train_data["label"]
n_dims = X_train.shape[1]
n_classes = np.unique(y_train).shape[0]
y_train = tf.keras.utils.to_categorical(y_train, num_classes=n_classes)
X_train.shape, y_train.shape

((42000, 784), (42000, 10))

In [6]:
model = Sequential()
model.add(Dense(n_dims, activation='relu'))
model.add(Dense(n_dims, activation='relu'))
model.add(Dense(n_classes, activation='softmax'))
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')
history = model.fit(X_train, y_train, validation_split=0.3, batch_size=128, epochs=10, verbose=2)

Epoch 1/10
230/230 - 4s - loss: 0.2774 - accuracy: 0.9170 - val_loss: 0.1464 - val_accuracy: 0.9555
Epoch 2/10
230/230 - 3s - loss: 0.0979 - accuracy: 0.9703 - val_loss: 0.0923 - val_accuracy: 0.9723
Epoch 3/10
230/230 - 3s - loss: 0.0558 - accuracy: 0.9822 - val_loss: 0.1011 - val_accuracy: 0.9697
Epoch 4/10
230/230 - 3s - loss: 0.0384 - accuracy: 0.9874 - val_loss: 0.1029 - val_accuracy: 0.9717
Epoch 5/10
230/230 - 3s - loss: 0.0290 - accuracy: 0.9907 - val_loss: 0.1176 - val_accuracy: 0.9679
Epoch 6/10
230/230 - 3s - loss: 0.0222 - accuracy: 0.9921 - val_loss: 0.0893 - val_accuracy: 0.9755
Epoch 7/10
230/230 - 3s - loss: 0.0174 - accuracy: 0.9946 - val_loss: 0.0982 - val_accuracy: 0.9752
Epoch 8/10
230/230 - 3s - loss: 0.0207 - accuracy: 0.9931 - val_loss: 0.1013 - val_accuracy: 0.9760
Epoch 9/10
230/230 - 4s - loss: 0.0127 - accuracy: 0.9954 - val_loss: 0.1289 - val_accuracy: 0.9722
Epoch 10/10
230/230 - 3s - loss: 0.0115 - accuracy: 0.9960 - val_loss: 0.1602 - val_accuracy: 0.9657

In [7]:
X_test = pd.read_csv('/kaggle/input/digit-recognizer/test.csv')
X_test.shape

(28000, 784)

In [8]:
predictions = model.predict(X_test)
labels = predictions.argmax(axis=1)
image_ids = np.arange(labels.shape[0]) + 1
labels.shape, image_ids.shape

((28000,), (28000,))

In [9]:
sample = pd.read_csv('/kaggle/input/digit-recognizer/sample_submission.csv')
submission = {'ImageId': image_ids,'Label': labels}
df = pd.DataFrame(submission, columns= ['ImageId', 'Label'])
df.to_csv ('/kaggle/working/digit-recognizer-submission-NN.csv', index = False, header=True)

Code following this line is an attempt to learn about CNNs from a Kaggle Notebook

In [58]:
X_train = X_train.values.reshape(-1, 28,28,1)
X_train.shape
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.1, random_state=2)
# In -> [[Conv2D->relu]*2 -> MaxPool2D -> Dropout]*2 -> Flatten -> Dense -> Dropout -> Out
# [Conv2D->relu]*2
model = Sequential()
model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', activation ='relu', input_shape = (28,28,1)))
model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', activation ='relu'))
# MaxPool2D
model.add(MaxPool2D(pool_size=(2,2)))

# Dropout
model.add(Dropout(0.25))

# x2
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.25))


model.add(Flatten())
model.add(Dense(256, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(10, activation = "softmax"))

model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

In [59]:
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)  # randomly flip images


datagen.fit(X_train)

In [61]:
history = model.fit(datagen.flow(X_train, y_train, batch_size=128),
                              epochs = 30, validation_data = (X_val, y_val),
                              verbose = 2, steps_per_epoch=X_train.shape[0] // 128)

Epoch 1/30
295/295 - 138s - loss: 0.2395 - accuracy: 0.9263 - val_loss: 0.0673 - val_accuracy: 0.9790
Epoch 2/30
295/295 - 138s - loss: 0.1408 - accuracy: 0.9584 - val_loss: 0.0479 - val_accuracy: 0.9864
Epoch 3/30
295/295 - 137s - loss: 0.1068 - accuracy: 0.9683 - val_loss: 0.0375 - val_accuracy: 0.9890
Epoch 4/30
295/295 - 138s - loss: 0.0868 - accuracy: 0.9740 - val_loss: 0.0360 - val_accuracy: 0.9895
Epoch 5/30
295/295 - 138s - loss: 0.0766 - accuracy: 0.9771 - val_loss: 0.0272 - val_accuracy: 0.9917
Epoch 6/30
295/295 - 138s - loss: 0.0682 - accuracy: 0.9799 - val_loss: 0.0312 - val_accuracy: 0.9893
Epoch 7/30
295/295 - 138s - loss: 0.0633 - accuracy: 0.9819 - val_loss: 0.0218 - val_accuracy: 0.9926
Epoch 8/30
295/295 - 137s - loss: 0.0591 - accuracy: 0.9818 - val_loss: 0.0280 - val_accuracy: 0.9905
Epoch 9/30
295/295 - 136s - loss: 0.0539 - accuracy: 0.9841 - val_loss: 0.0315 - val_accuracy: 0.9893
Epoch 10/30
295/295 - 137s - loss: 0.0542 - accuracy: 0.9837 - val_loss: 0.0232 - 

In [64]:
X_test = X_test.values.reshape(-1, 28,28,1)
predictions = model.predict(X_test)
labels = predictions.argmax(axis=1)
image_ids = np.arange(labels.shape[0]) + 1
labels.shape, image_ids.shape
sample = pd.read_csv('/kaggle/input/digit-recognizer/sample_submission.csv')
submission = {'ImageId': image_ids,'Label': labels}
df = pd.DataFrame(submission, columns= ['ImageId', 'Label'])
df.to_csv ('/kaggle/working/digit-recognizer-submission-CNN.csv', index = False, header=True)